# Tutorial 1: Forward and Inverse dynamics of a double pendulum
In this tutorial we will look at the dynamics of double pendulums, their properties, and some simple ways of exploiting them.


**Pre-requisites**

Knowledge of kinematic and dynamic models of simple robots and basic knowledge of integration schemes.

**Goals**

Exploring the modelling the dynamics of a double pendulum.

This notebook is organized as follows:

    1. Recap: Lagrangian Derivation of Equations of Motion (EOM)
    2. Implement the solution to the Forward and Inverse Dynamics
    3. Simulation using forward dynamics
    4. Gravity Compensation Control and Feedforward Control using Inverse Dynamics

Run the next cell to make sure you have all the necessary dependencies installed.

In [ ]:
# install dependencies
!pip install -r requirements.txt

In [ ]:
# imports
import numpy as np
import pandas as pd

from double_pendulum.model.plant import DoublePendulumPlant
from double_pendulum.simulation.simulation import Simulator
from double_pendulum.utils.plotting import plot_timeseries

%matplotlib inline
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.writer'] = "pillow"
import matplotlib.pyplot as plt

#from matplotlib import animation

<img src="double_pendulum_coords.png" width="50%" align="" />

### System Description

Our double pendulum has the following system parameters:

- Masses $m_1$ and $m_2$
- Lengths $l_1$ and $l_2$
- Center of masses $r_1$ and $r_2$
- Inertias $I_1$ and $I_2$
- Damping 

Additionally, gravity $g$ acts on the masses.

All in SI units.

The following parameters have been approximated using a system identification algorithm. Make sure to replace the motor ids in the first line of the next cell with your own to be able to run experiments on the real hardware.

In [ ]:
motors = [403, 379] # REPLACE WITH YOUR MOTOR IDS [SHOULDER, ELBOW]
mass = [0.10548177618443695, 0.07619744360415454]
length = [0.05, 0.05] 
com = [0.05, 0.03670036749567022]
inertia = [0.00046166221821039165, 0.00023702395072092597]
damping = [7.634058385430087e-12, 0.0005106535523065844]
#torque_limit = [np.inf, np.inf]
gravity = 9.81
torque_limit = [0.15, 0.15]
coulomb_fric = [0.00305,0.0007777]
x0 = [0.0, 0.0, 0.0, 0.0] 

## 1. Recap: Lagrangian Derivation of Equations of Motion (EOM)

The manipulator equation describes the inverse dynamics of the double pendulum

$$ M(\mathbf{q})\ddot{\mathbf{q}} + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} + G(\mathbf{q}) +F(\dot{\mathbf{q}}) = \mathbf{\tau} $$

with the vectors

$$ \mathbf{q} = (q_1, q_2)^T, \; \mathbf{x} = (\mathbf{q}, \dot{\mathbf{q}})^T, \; \mathbf{\tau} = (\tau_1, \tau_2)^T $$

We can obtain the terms of this equation using the Lagrangian Derivation. First, we calculate the potential ($V$) and kinetic ($K$) energies of the system as a function of its state. Their difference is the Lagrangian:

$$
L = K-V
$$

The Euler-Lagrange equations can be used to derive the Equations of Motion (EOM) of the system:
$$
\frac{d}{dt}\frac{\partial L}{\partial \dot{\mathbf{q}}} - \frac{\partial L}{\partial \mathbf{q}} = \tau
$$

The terms of the manipulator equation can then be isolated. The mass matrix is given by (with $s_1 = \sin(q_1), c_1 = \cos(q_1), \ldots$):
$$
  M =
    \left[ {\begin{array}{cc}
          I_1 + I_2 + l_1^2m_2 + 2l_1m_2r_2c_2 + g_r^2I_r + I_r &   I_2 + l_1m_2r_2c_2 \\
          I_2 + l_1m_2r_2c_2 &   I_2 \\
    \end{array}} \right]
$$

The Coriolis Matrix is:
$$
    C =
    \left[\begin{matrix}
    - 2 \dot{q}_2 l_{1} m_{2} r_{2} s_2 & - \dot{q}_2 l_{1} m_{2} r_{2} s_2\\
    \dot{q}_1 l_{1} m_{2} r_{2} s_2 &   0
    \end{matrix}\right]
$$

and the gravity vector:
$$
    G =
    \left[\begin{matrix}
    - g m_{1} r_{1} s_1 - g m_{2} \left(l_{1}
s_1 + r_{2} s_{1+2} \right)\\
    - g m_{2} r_{2} s_{1+2}
    \end{matrix}\right]
$$

and the friction term:
$$
    F = 
    \left[\begin{matrix}
    b \dot{q}_1 + \mu \arctan(100 \dot{q}_1)\\
    b \dot{q}_2 + \mu \arctan(100 \dot{q}_2)
    \end{matrix}\right]
$$
#### Equilibrium points

An equilibrium point is that in which the system will remain if it is undisturbed. For our system, this means that the joint velocities and accelerations must stay at zero. For this assesment we will also assume negligible function. We can plug this information into the EOM to find these configurations:

$$
G(\mathbf{q}) = 0
$$

The result is four equilibrium points. The joint coordinates at these points are:

- $(0,0)$
- $(\pi,0)$
- $(0,\pi)$
- $(\pi,\pi)$

Note that only the first point in the list is stable, meaning that, for the other ones, even a minimal disturbance will eventually drive them away from the equilibrium point.

## 2. Implementation of the solution to the Forward and Inverse Dynamics

For our simulations, as well as some control strategies, we will need to include the knowledge of the system dynamics in the pendulum plant. Complete the following function, which will provide the inverse dynamics of the system. Remember that the solution to the inverse dynamics problem provides the torque that needs to be provided to achieve a set acceleration. Be careful to include the contribution of the friction, which acts directly on the joints.

In [ ]:
def inverse_dynamics(self, x, acc):

    M = self.mass_matrix(x)
    C = self.coriolis_matrix(x)
    G = self.gravity_vector(x)
    F = self.coulomb_vector(x)

    ## Type here!
    
    tau = np.zeros([2,])
    
    ##
    
    return tau

DoublePendulumPlant.inverse_dynamics = inverse_dynamics

Now, do the same for the forward dynamics, which provide the acceleration with which a system responds to a torque input. This function is also used for the simulation of the double pendulum. Complete it so that we can use it to predict the evolution of the system.

In [ ]:
def forward_dynamics(self, x, tau):
    # pos = np.copy(x[:self.dof])
    vel = np.copy(x[self.dof :])

    M = self.mass_matrix(x)
    C = self.coriolis_matrix(x)
    G = self.gravity_vector(x)
    F = self.coulomb_vector(x)

    ## Type here!
    
    accn = np.zeros([2,])
    
    ##
    return accn

DoublePendulumPlant.forward_dynamics = forward_dynamics

## 3. Simulation using forward dynamics

To simulate the system forward in time, we have the system dynamics in the following form:

$$
\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, \mathbf{u})
$$

where $\mathbf{x}$ and $\mathbf{u}$ are the state and the input, respectively. In the previous expression, $\mathbf{f}$ is the forward dynamics function. Note that what we have is a first-order ODE.
To simulate the system we require a relation that can approximate the following:

$$
\mathbf{x_{i+1}} = \mathbf{x_{i}} + \int_{t_0}^{t_f} \mathbf{f}(\mathbf{x_{i}}, \mathbf{u_{i}}) dt
$$

For this, we will use a numerical integration scheme. With it, we will predict the state at the next instant, $\mathbf{x_{i+1}}$, as a function of the state at the current instant, $\mathbf{x_{i}}$, the time passed between the two, $h$, and the current control input, $\mathbf{u_i}$.

One extremely simple numerical integration scheme is the Forward Euler Method.

$$
\mathbf{x_{i+1}} = \mathbf{x_i} + h\mathbf{f}(\mathbf{x_i}, \mathbf{u_i})
$$

Implement the Forward Euler Method in the next cell to complete the simulation:

In [ ]:
def euler_scheme(self, t, x, h, u):

    f = self.plant.rhs(t,x,u)
    ## Type here!
    xnew = np.zeros([4,])
    ##
    return xnew

Simulator.euler_scheme = euler_scheme

Run the next cell to create an instance of the *DoublePendulumPlant* class and an instance of the *Simulator* class. The double pendulum is created using the parameters defined earlier. After instantiating the double pendulum plant, it is used to instantiate the simulator class. If everything is correctly set up, after a few seconds, you should see an animation of the double pendulum swinging without external power.

In [ ]:
dt = 0.002
t_final = 10.0
x0 = [np.pi / 2.0, -np.pi / 2.0, 0.0, 0.0]

plant = DoublePendulumPlant(
    mass=mass,
    length=length,
    com=com,
    gravity=gravity,
    inertia=inertia,
    damping=damping,
    torque_limit=torque_limit,
    coulomb_fric=coulomb_fric
)

sim = Simulator(plant=plant)

T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=t_final,
    dt=dt,
    controller=None,
    integrator="euler",
)

html = HTML(anim.to_jshtml())
display(html)
plt.close()

Let's see how accurate this simulation is. To connect to the real pendulum, run the next cell. Only run the next cell once.

When you run the next cell, lift the pendulum and let it fall. Once you are not touching it anymore, press enter. The result should be a simulation and the evolution of the real system from the same initial conditions.

In [ ]:
Treal, Xreal, Ureal, U_desreal = sim.run_experiment(
    tf=t_final,
    dt=dt,
    controller=None,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow],
)

x_init = Xreal[0].copy()
Tsim, Xsim, Usim = sim.simulate(
    t0=0.0,
    x0=x_init,
    tf=t_final,
    dt=dt,
    controller=None,
    integrator="runge_kutta",
)

plot_timeseries(
    T=Tsim,
    X=Xsim,
    U=Usim,
    T_des=Treal,
    X_des=Xreal,
    U_des=Ureal,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

Is the simulation accurate? What can you observe about the nature of the system from the comparison?

#### PD control

To further test our simulation and hardware, we will introduce a PD controller. This controller provides an input that drives the system to minimize the error with respect to a setpoint. It generally takes the shape of the following control law:

$$
u(t) = -\left[ \begin{array}{cc} K_p & K_d\end{array}\right] \mathbf{\bar{x}}
$$

where $\mathbf{\bar{x}}$ is the difference between our setpoint and our current state. The next cell imports and creates the instance of a PID controller class. It also defines the setpoint and the gains of the controller. To turn it into a PD, we simply get rid of the integral term.

In [ ]:
from double_pendulum.controller.pid.point_pid_controller import PointPIDController

x0 = [0.0, 0.0, 0.0, 0.0]                                                                                                                                                              
goal = [np.pi, 0.0, 0.0, 0.0]
torque_limit = [0.15, 0.15]

Kp = 0.14
Kd = 0.002

pid_controller = PointPIDController(torque_limit=torque_limit, dt=dt)
pid_controller.set_parameters(Kp=Kp, Ki=0, Kd=Kd)
pid_controller.set_goal(goal)
pid_controller.init()

Run the next cell to simulate the controller on the double pendulum.

In [ ]:
sim.reset()
T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=t_final,
    dt=dt,
    controller=pid_controller,
    integrator="runge_kutta",
)

html = HTML(anim.to_jshtml())
display(html)
plt.close()

plot_timeseries(
    T,
    X,
    U,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

Now, if the simulation worked, you may try this controller on the real system. While connected to the real pendulum, run the cell below to start the experiment. Try to induce disturbances by pushing it and see how the system reacts. You may also update the gains $K_p$ and $K_d$ to tune the controller.

In [ ]:
T, X, U, U_des = sim.run_experiment(
    tf=t_final,
    dt=dt,
    controller=pid_controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow],
)

plot_timeseries(
    T=T,
    X=X,
    U=U,
    T_des=T,
    U_des=U_des,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

## 4. Gravity Compensation Control and Feedforward Control using Inverse Dynamics

#### Gravity Compensation

The knowledge of the dynamics of the double pendulum allows us to create controllers that compute the torque needed to achieve certain motions. One of these controllers is the gravity compensation controller. The goal of this controller is to maintain the position of the pendulum by counteracting gravity.

This can be achieved by providing the following torque:

$$
\tau(\mathbf{x}) = G(\mathbf{q})
$$

So that the EOM are left as $M(\mathbf{q})\ddot{\mathbf{q}} + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} = 0$ $\rightarrow$ System without forces

Complete the *GravityCompensationController* class so that this effect is achieved:

In [ ]:
from double_pendulum.controller.abstract_controller import AbstractController

class GravityCompensationController(AbstractController):
    def __init__(
        self,
        mass=[0.5, 0.6],
        length=[0.3, 0.2],
        com=[0.3, 0.2],
        damping=[0.1, 0.1],
        coulomb_fric=[0.0, 0.0],
        gravity=9.81,
        inertia=[None, None],
        torque_limit=[0.0, 1.0],
    ):
        self.mass = mass
        self.length = length
        self.com = com
        self.damping = damping
        self.cfric = coulomb_fric
        self.gravity = gravity
        self.inertia = inertia
        self.torque_limit = torque_limit

        self.plant = DoublePendulumPlant(
            mass=self.mass,
            length=self.length,
            com=self.com,
            damping=self.damping,
            gravity=self.gravity,
            coulomb_fric=self.cfric,
            inertia=self.inertia,
            torque_limit=self.torque_limit,
        )

    def get_control_output(self, x, t=None):
        g = self.plant.gravity_vector(x)

        ## Type here!
        u = np.zeros([2,])
        ##

        # Limits on allowed torque, do not modify
        u[0] = np.clip(u[0], -self.torque_limit[0], self.torque_limit[0]) 
        u[1] = np.clip(u[1], -self.torque_limit[1], self.torque_limit[1])
        
        return u

Now, create an instance of the controller class and test it in a simulation.

In [ ]:
gravcomp_controller = GravityCompensationController(
    mass=[mass[0], mass[1]],
    length=length,
    com=com,
    damping=damping,
    gravity=gravity,
    inertia=inertia,
    torque_limit=torque_limit,
)

x0 = [np.pi / 2.0, np.pi / 4.0, 0.0, 0.0]

sim.reset()
T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=t_final,
    dt=dt,
    controller=gravcomp_controller,
    integrator="runge_kutta",
)

html = HTML(anim.to_jshtml())
display(html)
plt.close()

plot_timeseries(
    T,
    X,
    U,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

Run the next cell to test the controller. Once it is working, try to manually change the configuration of the double pendulum, does it hold the new position?

In [ ]:
T, X, U, U_des = sim.run_experiment(
    tf=t_final,
    dt=dt,
    controller=gravcomp_controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow],
)

plot_timeseries(
    T=T,
    X=X,
    U=U,
    T_des=T,
    U_des=U_des,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

#### PD + Gravity compensation

Now that we have our gravity compensation controller working, we can combine it with a PD controller. This controller combines the nullifying of the effect of gravity with the PD controller to achieve smoother and more measured inputs. The control law can be stated as folows:

$$
u(t) = G(\mathbf{q}) + \left[ \begin{array}{cccc} K_p & 0 & K_d & 0 \\ 0 & K_p & 0 & K_d
\end{array} \right] \mathbf{\bar{x}}
$$

Run the two cells below to simulate this new controller. How does it compare to the previous PD controller?

In [ ]:
from double_pendulum.controller.gravity_compensation.PID_gravity_compensation_controller import PIDGravityCompensationController

Kp = 0.14
Ki = 0.0
Kd = 0.002

gravcompPID_controller = PIDGravityCompensationController(
    mass=mass,
    length=length,
    com=com,
    damping=damping,
    gravity=gravity,
    inertia=inertia,
    torque_limit=torque_limit,
    dt=dt,
)
gravcompPID_controller.set_parameters(Kp=Kp, Ki=Ki, Kd=Kd)
gravcompPID_controller.set_goal(goal)

Run the next cell to simulate the controller

In [ ]:
x0 = [0.0, 0.0, 0.0, 0.0]

sim.reset()
sim.set_desired_state(goal)

gravcompPID_controller.init()
T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=t_final,
    dt=dt,
    controller=gravcompPID_controller,
    integrator="runge_kutta",
)

html = HTML(anim.to_jshtml())
display(html)
plt.close()

plot_timeseries(
    T,
    X,
    U,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

Now you may try the controller on the real system. Run the cell below to start the experiment. Try playing around with the gains of the controller. How does the behavior of the controller differ from the previous controller when different gains are used?

In [ ]:
T, X, U, U_des = sim.run_experiment(
    tf=t_final,
    dt=dt,
    controller=gravcompPID_controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow],
)

plot_timeseries(
    T=T,
    X=X,
    U=U,
    T_des=T,
    U_des=U_des,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)